In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.decomposition import PCA
from scipy.io import loadmat
from scipy.sparse import vstack
import os
from scipy.optimize import curve_fit

Archive:  Savi.zip
  inflating: spikesI_finalRND.npy    
  inflating: spikesI_finalSF1_RND.npy  
  inflating: spikesI_finalSF2_RND.npy  
  inflating: spikesI_finalSF2_SF2.npy  
  inflating: spikesI_finalSF3_RND.npy  
  inflating: spikesI_finalSW.npy     
  inflating: spikesI_p03.npy         
  inflating: spikesT_finalRND.npy    
  inflating: spikesT_finalSF1_RND.npy  
  inflating: spikesT_finalSF2_RND.npy  
  inflating: spikesT_finalSF2_SF2.npy  
  inflating: spikesT_finalSF3_RND.npy  
  inflating: spikesT_finalSW.npy     
  inflating: spikesT_p03.npy         
Archive:  topologies.zip
   creating: sources/
  inflating: sources/source_RND.npy  
  inflating: sources/source_RND_19.npy  
  inflating: sources/source_RND_RND.npy  
  inflating: sources/source_SF1_RND.npy  
  inflating: sources/source_SF2_RND.npy  
  inflating: sources/source_SF2_SF2.npy  
  inflating: sources/source_SF3_RND.npy  
  inflating: sources/source_SW.npy   
   creating: targets/
  inflating: targets/target_RND.npy  

In [ ]:
# Initialize lists to store PCA results
cum_vars = []
weights = []

# Load spike timestamps and corresponding neuron indices
timestamps, indices = np.load('spikesT.npy'), np.load('spikesI.npy')

# Number of channels (neurons)
numCh = max(indices) + 1

# Total acquisition time (in seconds, rounded up)
acqTime = np.ceil(max(timestamps))

# Create a list to store spike times (in ms) for each neuron
peak_trains = [[] for _ in range(numCh)]
for time, neuron in zip(timestamps, indices):
    peak_trains[neuron].append(int(time * 1000))

# Compute instantaneous firing rate (IFR) for each neuron
ifr_array = []
for peaks in peak_trains:
    # Initialize binary spike train
    peak_train = np.zeros(int(acqTime * 1000))
    peak_train[peaks] = 1

    # Convolve with a 10 ms window to estimate firing rate
    ifr = np.convolve(peak_train, np.ones(10), mode='same')
    ifr_array.append(ifr)

# Stack IFRs into a time × neuron matrix
ifr_array = np.vstack(ifr_array).T

# Perform PCA across neurons
pca = PCA(n_components=numCh)
principal_components = pca.fit_transform(ifr_array)

# Extract explained variance ratio for each component
explained_variance = pca.explained_variance_ratio_

# Compute cumulative explained variance (in %)
cumulative_variance = np.cumsum(explained_variance) * 100

# Store results
cum_vars.append(cumulative_variance)
weights.append(pca.components_)

In [ ]:
# Plot cumulative explained variance (single simulation)

fig, ax = plt.subplots(figsize=(15.9/2.56, 10.6/2.56))
color = '#d90429'  # dark red

# Extract the cumulative variance from the single simulation
cum_var = cum_vars[0]

# Print contribution of the first two components
print(f'1st = {cum_var[0]:.1f} %, 2nd = {(cum_var[1]-cum_var[0]):.1f} %')

# Plot cumulative explained variance curve
ax.plot(range(numCh + 1), [0] + cum_var.tolist(),
        marker='o', markersize=2, color=color, linewidth=1)

# Compute the area under the curve (AUC)
auc = np.sum(cum_var / 100) / numCh
# Optional: reference diagonal line
# ax.plot([0, numCh], [0, 100], linestyle='--', color='k')

# Axis limits and ticks
ax.set_ylim(-2, 102)
ax.set_xlim(0, numCh)
ax.set_xticks(range(0, numCh + 1, 10))
ax.set_xticklabels(range(0, numCh + 1, 10))
ax.tick_params(labelsize=6)

# Axis labels
fig.supxlabel('# of principal components', fontsize=8)
fig.supylabel('Cumulative explained variance (%)', fontsize=8)

fig.tight_layout()
plt.show()

# Structure vs Dynamics

In [ ]:
from scipy.stats import spearmanr
from scipy.optimize import curve_fit
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np

# Define the half-sigmoid fitting function
def half_sigmoid(x, a, b, c):
    return a / (b + np.exp(-c * x)) - 0.5

# Load connectivity data (sources and targets of each synapse)
sources = np.load('source_SF2_RND.npy')
targets = np.load('target_SF2_RND.npy')

# Select PCA weights from the single simulation
weight = weights[0]

# ---- INDEGREE ANALYSIS ----
print("Indegree analysis")

# Compute indegree: number of incoming connections for each neuron
unique_in, indegree_count = np.unique(targets, return_counts=True)

# Sort by indegree
sorted_indices = np.argsort(indegree_count)
sorted_indegree = indegree_count[sorted_indices]
sorted_weight = weight[0][sorted_indices]  # use weights from the first PCA component
nbins = 30

# Attempt to fit the half-sigmoid function
try:
    popt, pcov = curve_fit(
        half_sigmoid,
        sorted_indegree,
        sorted_weight,
        bounds=(0, np.inf),
        p0=[1, 1, 1]
    )
    predicted_weight = half_sigmoid(sorted_indegree, *popt)
    ss_res = np.sum((sorted_weight - predicted_weight) ** 2)
    ss_tot = np.sum((sorted_weight - np.mean(sorted_weight)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    fitting_flag = True
except:
    fitting_flag = False

# Spearman correlation between indegree and PCA weight
rho_in = spearmanr(sorted_indegree, sorted_weight)[0]
print(f"Spearman coefficient (IN): {rho_in:.4f}")

# Create a figure with main scatter + histograms
fig = plt.figure(figsize=(7.95/2.54, 5.3/2.54))
gs = GridSpec(4, 4, figure=fig)
colors = ['#D72638', '#8B0000', '#FF6F61']

# Main scatter plot
ax_main = fig.add_subplot(gs[1:, :-1])
ax_main.scatter(sorted_indegree, sorted_weight, s=50, color=colors[0], alpha=0.66)

# Plot fitted curve if available and good quality (R² > 0.95)
if fitting_flag and r2 > 0.95:
    indegree_range = np.linspace(np.min(sorted_indegree), np.max(sorted_indegree), 1000)
    ax_main.plot(indegree_range, half_sigmoid(indegree_range, *popt), color='k', label=rf'$r_2$ = {r2:.2f}')

# Main axis settings
ax_main.set_xlabel(r'$deg^{IN}$', fontsize=10)
ax_main.set_ylabel(r'$w^{PCA}$', fontsize=10)
ax_main.set_xlim(0, 100)
ax_main.set_ylim(-0.02, 0.25)
ax_main.tick_params(labelsize=8)

# Top histogram (indegree distribution)
ax_xhist = fig.add_subplot(gs[0, :-1], sharex=ax_main)
ax_xhist.hist(sorted_indegree, bins=nbins, color=colors[1])
ax_xhist.axis("off")

# Right histogram (PCA weight distribution)
ax_yhist = fig.add_subplot(gs[1:, -1], sharey=ax_main)
ax_yhist.hist(sorted_weight, bins=nbins, orientation='horizontal', color=colors[2])
ax_yhist.axis("off")

fig.tight_layout()
plt.show()
# fig.savefig('indegree_single_sim.tif', format="tiff", dpi=600)


# ---- OUTDEGREE ANALYSIS ----
print("Outdegree analysis")

# Compute outdegree: number of outgoing connections for each neuron
unique_out, outdegree_count = np.unique(sources, return_counts=True)

# Sort by outdegree
sorted_indices = np.argsort(outdegree_count)
sorted_outdegree = outdegree_count[sorted_indices]
sorted_weight = weight[0][sorted_indices]
nbins = 30

# Attempt to fit the half-sigmoid function
try:
    popt, pcov = curve_fit(
        half_sigmoid,
        sorted_outdegree,
        sorted_weight,
        bounds=(0, np.inf),
        p0=[1, 1, 1]
    )
    predicted_weight = half_sigmoid(sorted_outdegree, *popt)
    ss_res = np.sum((sorted_weight - predicted_weight) ** 2)
    ss_tot = np.sum((sorted_weight - np.mean(sorted_weight)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    fitting_flag = True
except:
    fitting_flag = False

# Spearman correlation between outdegree and PCA weight
rho_out = spearmanr(sorted_outdegree, sorted_weight)[0]
print(f"Spearman coefficient (OUT): {rho_out:.4f}")

# Create a figure with main scatter + histograms
fig = plt.figure(figsize=(7.95/2.54, 5.3/2.54))
gs = GridSpec(4, 4, figure=fig)
colors = ['#4A90E2', '#002F6C', '#B3D4FC']

# Main scatter plot
ax_main = fig.add_subplot(gs[1:, :-1])
ax_main.scatter(sorted_outdegree, sorted_weight, s=50, color=colors[0], alpha=0.66)

# Plot fitted curve if available and good quality
if fitting_flag and r2 > 0.95:
    outdegree_range = np.linspace(np.min(sorted_outdegree), np.max(sorted_outdegree), 1000)
    ax_main.plot(outdegree_range, half_sigmoid(outdegree_range, *popt), color='k', label=rf'$r_2$ = {r2:.2f}')

# Main axis settings
ax_main.set_xlabel(r'$deg^{OUT}$', fontsize=10)
ax_main.set_ylabel(r'$w^{PCA}$', fontsize=10)
ax_main.set_xlim(0, 100)
ax_main.set_ylim(-0.02, 0.25)
ax_main.tick_params(labelsize=8)

# Top histogram (outdegree distribution)
ax_xhist = fig.add_subplot(gs[0, :-1], sharex=ax_main)
ax_xhist.hist(sorted_outdegree, bins=nbins, color=colors[1])
ax_xhist.axis("off")

# Right histogram (PCA weight distribution)
ax_yhist = fig.add_subplot(gs[1:, -1], sharey=ax_main)
ax_yhist.hist(sorted_weight, bins=nbins, orientation='horizontal', color=colors[2])
ax_yhist.axis("off")

fig.tight_layout()
plt.show()